In [ ]:
!pip install PyMuPDF --quiet

In [ ]:
# Mount the drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

import fitz  # PyMuPDF library

def extract_text_by_columns(pdf_path):
    with fitz.open(pdf_path) as pdf_file:
        text = ""
        for page in pdf_file:
            texts = page.get_text("blocks")
            for block in texts:
                bbox = block[0]  # Bounding box of the block
                if bbox < 300:  # Arbitrary limit to separate columns
                    text += block[4]
                else:
                    text += block[4]
            text += "\n"  # Add newline after each page
            text = text.replace("\r", "")
            text = text.replace("\x17", "")
            text = text.replace("V I R G I N I A  D R I V E R ’ S  M A N U A L", "")
            text = text.replace("|", "")
            text = text.replace("   ", "")
            text = text.replace("\n", "")
    return text


# Example usage
pdf_path = r'/content/drive/MyDrive/Colab Notebooks/DMV_test.pdf'
pdf_content = extract_text_by_columns(pdf_path)
# pdf_content

Mounted at /content/drive


# Using distilbert model


In [ ]:
from transformers import pipeline, AutoModelForQuestionAnswering, AutoTokenizer

# Loading the pre-trained model and tokenizer
model_name = "distilbert-base-cased-distilled-squad"
# model_revision = "626af31"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create the question answering pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

# context = pdf_content
context = '''Speed Limit: These signs tell you the maximum legal speed that you may drive on the road where the sign is posted when weather conditions are good. Some roads have electronic speed limit signs that change based on weather or traffic conditions. During rain, snow and ice, you may receive a ticket for driving too fast for the conditions even if you are driving at or less than the posted speed limit. Do Not Enter – Wrong Way: These signs mean you cannot drive in that direction. If you drive past these signs you are going in the wrong direction and could get into a head-on crash with vehicles headed your way. Immediately slow down, pull over, and cautiously turn around.One Way: Traffic flows only  in the direction of the arrow.No Left Turn: Left turns are against the law.'''


In [ ]:
def perform_qa(question, context):
    result = qa_pipeline(question=question, context=context)
    return result

In [ ]:
question = ["What is speed limit?"]

for ques in question:
    print(f"Question: {ques}")
    answer = perform_qa(ques, context)
    print(f"Answer: {answer['answer']}")
    print()

Question: What is speed limit?
Answer: maximum legal speed



# Using T5 model

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

# Load pre-trained T5 model and tokenizer
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


In [ ]:
question = "What is speed limit?"
# context = '''Speed Limit: These signs tell you the maximum legal speed that you may drive on the road where the sign is posted when weather conditions are good. Some roads have electronic speed limit signs that change based on weather or traffic conditions. During rain, snow and ice, you may receive a ticket for driving too fast for the conditions even if you are driving at or less than the posted speed limit. Do Not Enter – Wrong Way: These signs mean you cannot drive in that direction. If you drive past these signs you are going in the wrong direction and could get into a head-on crash with vehicles headed your way. Immediately slow down, pull over, and cautiously turn around.One Way: Traffic flows only  in the direction of the arrow.No Left Turn: Left turns are against the law.'''
context = pdf_content

In [ ]:
# Tokenize inputs
inputs = tokenizer.encode("question: " + question + " context: " + context, return_tensors="pt")
print(inputs.shape)

outputs = model.generate(inputs, max_length=100)

answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Answer:", answer)

torch.Size([1, 2617])
Answer: maximum legal speed
